In [40]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml import Pipeline
from pyspark.sql import functions as f
from pyspark.ml.functions import vector_to_array
from mleap.pyspark.spark_support import SimpleSparkSerializer


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
%%configure -f
{
    "conf": {
        "spark.jars.packages": "ml.combust.mleap:mleap-spark_2.12:0.20.0,ml.combust.mleap:mleap-spark-base_2.12:0.20.0",
        "spark.pyspark.python": "python3",
        "spark.pyspark.virtualenv.enabled": "true",
        "spark.pyspark.virtualenv.type": "native",
        "spark.pyspark.virtualenv.bin.path": "/usr/bin/virtualenv"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1709942347767_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1709942347767_0002,pyspark,idle,Link,Link,None,✔


In [39]:
sc.install_pypi_package("mleap==0.20.0")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
df = spark.read.csv("s3://sagemaker-us-east-1-381492271173/heart/data/train/train.csv", inferSchema=True)

 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
columnas_numericas = df.columns[1:]
columnas_numericas
 
ensamblador = VectorAssembler(inputCols=columnas_numericas, outputCol="features")
escalador = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)
pipeline = Pipeline(stages=[ensamblador, escalador])
 
pipeline_model = pipeline.fit(df)
df_scalado = pipeline_model.transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
df_scalado.select("_c0", "scaled_features").repartition(1).write.parquet("s3://sagemaker-us-east-1-381492271173/heart/data/train/train_scaled.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
df_s = (df_scalado.withColumn("xs", vector_to_array("scaled_features"))).select(["_c0"] + [f.col("xs")[i] for i in range(13)])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
df_s.repartition(1).write.mode("overwrite").parquet("s3://sagemaker-us-east-1-381492271173/heart/data/train/train_scaled.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
df_test = spark.read.csv("s3://sagemaker-us-east-1-381492271173/heart/data/test/test.csv", inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
df_scalado_test = pipeline_model.transform(df_test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
df_s_test = (df_scalado_test.withColumn("xs", vector_to_array("scaled_features"))).select(["_c0"] + [f.col("xs")[i] for i in range(13)])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
df_s_test.repartition(1).write.mode("overwrite").parquet("s3://sagemaker-us-east-1-381492271173/heart/data/test/test_scaled.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
pipeline_model.serializeToBundle("jar:file:/tmp/model.zip", df_scalado_test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
import zipfile

with zipfile.ZipFile("/tmp/model.zip") as zf:
    zf.extractall("/tmp/model")

import tarfile

with tarfile.open("/tmp/model.tar.gz", "w:gz") as tar:
    tar.add("/tmp/model/bundle.json", arcname="bundle.json")
    tar.add("/tmp/model/root", arcname="root")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
sc.install_pypi_package("boto3")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
import boto3

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
import os
import boto3

bucket = 'sagemaker-us-east-1-381492271173'
s3 = boto3.resource("s3")
file_name = os.path.join("heart/pyspark-model", "model.tar.gz")
s3.Bucket(bucket).upload_file("/tmp/model.tar.gz", file_name)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/mnt1/yarn/usercache/livy/appcache/application_1709942347767_0002/container_1709942347767_0002_01_000001/tmp/1709947448447-0/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)